
$\qquad$ $\qquad$ $\qquad$  **TDA206/DIT206 Discrete Optimization: Home Assignment 2 -- Integer LP and Relaxation** <br />
$\qquad$ $\qquad$ $\qquad$                   **Grader: Marc Constantin** <br />
$\qquad$ $\qquad$ $\qquad$                   **Due Date: 17th Feb** <br />
$\qquad$ $\qquad$ $\qquad$                   **Submitted by: Name, Personal No., Email** <br />


---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook, and equations wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook do not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Question 1.
(5 points) 
There are 4 space colonies, each of which  requires a certain number of plasma conduits. There are 3 starbases in the vicinity. Each of them has total number of conduits they can spare and supply to the colonies. For each pair of starbase and colony, there is an associated cost for sending a cargo ship  (each of which carries one plasma conduit), as shown in the table below:


\begin{array}{l|c|c|c|c|c} 
      & Triacus & New Berlin  & Strnad  & Vega  & supply\\ \hline
 Farpoint &   6 &  9 & 10 & 8 & 35\\
 Yorktown &  9 & 5 & 16 & 14 & 40\\
 Earhart & 12 &  7 & 13 & 9 & 50\\ \hline
    demand & 20 &30&30&45& \left(\sum=125\right) \\ 
\end{array}

Your goal is to supply the colonies the plasma conduits they need, at minimum cost.


Formulate a LP to solve the problem, explain why the solution is integral with a proof.

In [12]:
import cvxpy as cp 

# Variables
xf = cp.Variable(4, nonneg=True)
xy = cp.Variable(4, nonneg=True)
xe = cp.Variable(4, nonneg=True)

# Objective function
obj = cp.Minimize(6 * xf[0] + 9 * xf[1] + 10 * xf[2] + 8 * xf[3] + 
                  9 * xy[0] + 5 * xy[1] + 16 * xy[2] + 14 * xy[3] + 
                  12 * xe[0] + 7 * xe[1] + 13 * xe[2] + 9 * xe[3])

# Constraints
constraints = []
# Supply
constraints.append(sum(xf) <= 35)
constraints.append(sum(xy) <= 40)
constraints.append(sum(xe) <= 50)
# Demand
t, n, s, v = zip(xf, xy, xe)
constraints.append(sum(t) >= 20)
constraints.append(sum(n) >= 30)
constraints.append(sum(s) >= 30)
constraints.append(sum(v) >= 45)

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {round(opt, 2)}")
print(f"Farpoint: {[round(float(i.value), 2) for i in xf]}")
print(f"Yorktown: {[round(float(i.value), 2) for i in xy]}")
print(f"Earhart: {[round(float(i.value), 2) for i in xe]}")

Objective value: 1020.0
Farpoint: [10.0, 0.0, 25.0, 0.0]
Yorktown: [10.0, 30.0, 0.0, 0.0]
Earhart: [0.0, 0.0, 5.0, 45.0]


# Question 2.

Recall the Minimum Weight Vertex Cover (VC) Problem: Given an undirected graph $G=(V, E)$, with node set $V$ and edge set $E$, where each node has a positive weight $w(v)$ associated with it (see the attached figure), the goal is to select a subset $V'\subseteq V$ of nodes such that every edge has at least one node incident to it, and the total selected node weight $\sum_{v\in V'} w(v)$ is minimized. 

* (4 points) Formulate the ILP for the VC problem for the attached graph, and solve it using **CVXPY** integer solver, for instance, `myVar = cp.Variable(<dim>, integer=True)`.
* (2 points) Pass to the LP relaxation and solve it using **CVXPY** and comment on the relation between the two solutions.
* (2 points) Apply the rounding rule discussed in class to the optimal LP solution to obtain a solution to the ILP and compare it to the optimal ILP solution.

<!-- ![title](vertex_cover_example.png) -->


In a Minimum Weight Vertex Cover we want to minimize the total cost of picking vertices under the constraints that all edges must be connected to one vertex in the chosen set.

We can formulate this problem as follows.

$ \text{min} \sum_{v \in V'}{w(v)} \\$
s.t.
$\\ x_u + x_v \geq 1, (u,v) \in E \\$
$x =
\begin{cases}
1, & u \in V'\\
0, & \text{otherwise}
\end{cases}$

In [13]:
# Function returning Minimum Weight Vertex Cover

def min_weight_VC(nodes, weights):
    # Objective function
    obj = cp.Minimize(sum(x * y for x, y in zip(nodes, weights)))

    # Constraints
    constraints = [nodes[0] + nodes[1] >= 1,
                   nodes[1] + nodes[2] >= 1,
                   nodes[1] + nodes[4] >= 1,
                   nodes[2] + nodes[3] >= 1,
                   nodes[2] + nodes[4] >= 1,
                   nodes[3] + nodes[4] >= 1,
                   nodes[4] + nodes[5] >= 1,
                   nodes >= 0
                ]

    # Solution
    prob = cp.Problem(obj, constraints)
    return prob.solve()

In [14]:
# ILP solution for the attached graph

# Variables
nodes = cp.Variable(6, integer=True)
weights = [1, 3, 4, 2, 2, 4]

opt = min_weight_VC(nodes, weights)
print(f"Objective value: {round(opt, 2)}")
print(f"Solution vector: {[round(float(n.value), 2) for n in nodes]}")

Objective value: 7.0
Solution vector: [-0.0, 1.0, -0.0, 1.0, 1.0, -0.0]


In [15]:
# Solution of the LP relaxation of the attached graph

# Variables (no constraint on them being integral)
nodes = cp.Variable(6)
weights = [1, 3, 4, 2, 2, 4]

opt = min_weight_VC(nodes, weights)
print(f"Objective value: {round(opt, 2)}")
print(f"Solution vector: {[round(float(n.value), 2) for n in nodes]}")

Objective value: 7.0
Solution vector: [0.48, 0.52, 0.48, 0.52, 1.0, -0.0]


Without integer constraints on the variables (nodes), they can be of any value. This solution does not make sense, because the node variables are supposed to be representations of booleans. We overcome this problem by rounding the nodes to 0 or 1:

$ \tilde{x_u} = 
\begin{cases}
1, & x_u^* \geq 0.5\\
0, & \text{otherwise}
\end{cases}
$

Then, we obtain the (relaxed) optimal solution by multiplying with nodes to the weights.

In this case, the optimal solution was identical to the ILP optimal, but this is not necessarily the case.

In [16]:
# Solution of the LP relaxation of the attached graph

# Variables (no constraint on them being integral)
nodes = cp.Variable(6)
weights = [1, 3, 4, 2, 2, 4]

opt = min_weight_VC(nodes, weights)

# round nodes to 0/1
nodes = [round(float(n.value)) for n in nodes]

# Update optimal value
new_opt = sum(x * y for x, y in zip(nodes, weights))

print(f"Objective value: {round(new_opt, 2)}")
print(f"Solution vector: {[n for n in nodes]}")

Objective value: 7
Solution vector: [0, 1, 0, 1, 1, 0]


# Question 3.

Consider a number of interpreters (Olof, Petra, Qamar,
  Rachel, Soren and Tao), as well as a set of languages (Arab,
  Bengali, Cantonese, Dutch, English, French and German). Each
  interpreter speaks a number of different languages (abbreviated by
  first letter), and has a certain per-diem integer cost:

\begin{array}{lll}
Interpreter & Languages & Cost\\
O & ABD & 3\\
P & C & 1\\
Q & CDG & 1\\
R & B & 2\\
S & G & 4\\
T & EF & 1\\
\end{array}

* (2 points) A *hypergraph* is a structure $H = (V,E)$ where $V$ is a set of vertices and $E$ is a collection of subsets of $V$. The special case when all subsets $e \in E$ have size exactly $2$ corresponds to the familiar case of a graph. A vertex cover in such a hypergraph is a subset $U \subseteq V$ such that $e \cap U \not = \emptyset$ for each $e \in E$ (note that this reduces to the usual vertex cover in graphs). Show that the problem of finding interpreters can be formulated as a vertex cover problem in a sutable hypergraph.
* (4 points) Develop a ILP formulation to finding the vertex cover of minimum cost in a hypergraph. The hypergraph can be represented as a $|V| \times |E|$ binary matrix $A$ where $A[i,j] = 1$ iff vertex $i$ is in edge $j$ and 0 otherwise. The costs for vertices are in an array $\texttt{c}$ where the cost of picking vertex $i$ is $c[i]$. Use the ILP formulation for the VC problem to hire the cheapest set of interpreters such that all languages are covered. Input the data above manuallly and solve it using **CVXPY**'s integer solver.
* (2 points) Pass to the LP relaxation and solve it using **CVXPY**.
* (2 points) Explain why the two solutions above are same (different).


Note: The problem of finding interpreters is interpreted as finding a set of interpreters s.t. all langauges are represented.

* Let $H=(V,E)$ where $V = \{O,P,Q,R,S,T\}$ and $E = \{(O), (O,R), (P,Q), (O,Q), (T), (T), (Q) \}$ where $e_i$ is the collection of interpreters speaking language $L_i$, $L = \{A,B,C,D,E,F,G\}$. A vertex cover in $H$ will select $U \subseteq V$ s.t. all languages are represented, thus solving the task.

In [42]:
import cvxpy as cp 

# Variables
interpreters = cp.Variable(6, integer=True)
costs = [3, 1, 1, 2, 4, 1]
# A = [[1, 1, 0, 1, 0, 0, 0],
#      [0, 0, 1, 0, 0, 0, 0],
#      [0, 0, 1, 1, 0, 0, 1],
#      [0, 1, 0, 0, 0, 0, 0],
#      [0, 0, 0, 0, 0, 0, 1],
#      [0, 0, 0, 0, 1, 1, 0]]
# print("A")
# print([A[row] for row in len(A)])

# Objective function
obj = cp.Minimize(sum(x * y for x, y in zip(interpreters, costs)))

# Constraints
# constraints = [interpreters >= 0]
# a_t = [[A[j][i] for j in range(len(A))] for i in range(len(A[0]))]

# for i in range(len(a_t)):
#     lan = []
#     for j in range(len(a_t[i])):
#         if a_t[i][j] == 1:
#             lan.append(j)
#     # constraints.append(sum(interpreters[k]) >= 1 for k in range(len(lan)))
#     s = []
#     for k in range(len(lan)):
#         s.append(interpreters[k])
#     constraints.append(sum(s) >= 1)

# print(f"Constraints: {constraints}")

constraints = [interpreters[0] >= 1,
               interpreters[0] + interpreters[3] >= 1,
               interpreters[1] + interpreters[2] >= 1,
               interpreters[0] + interpreters[2] >= 1, 
               interpreters[5] >= 1, 
               interpreters[2] + interpreters[4] >= 1,
               interpreters >= 0]

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {round(opt, 2)}")
print(f"Solution vector: {[float(i.value) for i in interpreters]}")

Objective value: 5.0
Solution vector: [1.0, -0.0, 1.0, -0.0, -0.0, 1.0]


In [45]:
import cvxpy as cp 

# Variables
interpreters = cp.Variable(6)
costs = [3, 1, 1, 2, 4, 1]

# Objective function
obj = cp.Minimize(sum(x * y for x, y in zip(interpreters, costs)))

# Constraints
constraints = [interpreters[0] >= 1,
               interpreters[0] + interpreters[3] >= 1,
               interpreters[1] + interpreters[2] >= 1,
               interpreters[0] + interpreters[2] >= 1, 
               interpreters[5] >= 1, 
               interpreters[2] + interpreters[4] >= 1,
               interpreters >= 0]

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {round(opt, 2)}")
print(f"Solution vector: {[round(float(i.value), 2) for i in interpreters]}")

Objective value: 5.0
Solution vector: [1.0, 0.0, 1.0, -0.0, -0.0, 1.0]


<!-- * (2 points) Explain why the two solutions above are same (different). -->
The ILP solution equals the LP relaxation solution. However, this is not due to that fractions of vertices can not be pat of an optimal solution, but rather due to the costs of the interpreters in the example. Since T is the only one speaking E and F, and O is the only one speaking A, we must choose those interpreters. Since O also speaks B and D, the remaining languages are C and G. Since Q speaks both C and G, and also has the lowest possible cost, there can not be any other optimal solution than choosing Q.

If we change the costs of Q to 2 and S to 1, we get the solution vector [1.0, 0.51, 0.49, 0.0, 0.51, 1.0].

In mathematical terms, our constraints say that $x_1 \geq 1$ and $x_6 \geq 1$ i.e. O and T is part of the solution. The constraints $x_1 + x_4 \geq 1$ and $x_1 + x_3 \geq 1$ is redundant since $x_1 \geq 1$ and $x_i \geq 0$ for all $x_i$. The remaining constraints are thus $x_2 + x_3 \geq 1$ and $x_3 + x_5 \geq 1$ where the objective function wants to minimize $x_2 + x_3 + 4 x_5$ which (regardless of ILP or LP) equals $x_3 = 1$. 

<!-- Since the objective function minimizes $\sum_{u \in V}{c_u x_u}$ and  -->


# Question 4. 
Consider the ILP and its LP relaxation corresponding to the VC problem for the graph $G$ given in the data file. This is a ***random graph*** $G(n,p)$ with $n=100$ vertices generated as follows: for each pair of vertices **independently**, we add an edge with probability $p=0.1$ (so the graph has about 1000 edges).

* **a**. (2 points) Find the optimal solution using **CVXPY**'s integer solver.
* **b**. (2 points) Solve the LP relaxation using **CVXPY** and apply the rounding rule discussed in class to obtain a vertex cover. Compare it to the optimal solution in part (a).
* **c**. (6 points) Consider the following rounding rule: we build up the vertex cover incrementally starting with $S:= \emptyset$. Now consider the edges in $G$ in any order. If an edge $(u,v)$ is already covered by a vertex in $S$, do nothing. Otherwise add to $S$ the vertex $u$ if $x^*(u) \geq x^*(v)$, or $v$ otherwise (where ${\bf x}^*$ is the LP optimum solution computed in part (b).  Comment why this also results in a vertex cover and has cost no more than that corresponding to the rounding rule in part (b). Compare the cost of the solution produced by this rule to the optimal solution.

In [63]:
# **a**. (2 points) Find the optimal solution using **CVXPY**'s integer solver.

import cvxpy as cp 

# Variables
nodes = cp.Variable(100, integer=True)

with open("graph.txt", "r") as f:
    graph = [[int(num) for num in line.split()] for line in f]

edges = []
for i in range(len(graph)):
    for j in range(len(graph[i])):
        if graph[i][j] == 1:
            edges.append((i,j))
print(f"Number of edges: {len(edges)}")

# Objective function
obj = cp.Minimize(sum(nodes))

# Constraints
constraints = [nodes >= 0]

for (m,n) in edges:
    constraints.append(nodes[m] + nodes[n] >= 1)

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {round(opt, 2)}")
print(f"Solution vector: {[round(float(i.value), 2) for i in nodes]}")

Number of edges: 980
Objective value: 70.0
Solution vector: [1.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0]


In [62]:
# (2 points) Solve the LP relaxation using **CVXPY** and apply the rounding rule discussed in class to obtain a vertex cover. Compare it to the optimal solution in part (a).

import cvxpy as cp 

# Variables
nodes = cp.Variable(100)

with open("graph.txt", "r") as f:
    graph = [[int(num) for num in line.split()] for line in f]

edges = []
for i in range(len(graph)):
    for j in range(len(graph[i])):
        if graph[i][j] == 1:
            edges.append((i,j))

# Objective function
obj = cp.Minimize(sum(nodes))

# Constraints
constraints = [nodes >= 0]

for (m,n) in edges:
    constraints.append(nodes[m] + nodes[n] >= 1)

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value (before rounding): {round(opt, 2)}")
print(f"Solution vector (before rounding): {[(float(i.value)) for i in nodes]}")

# round nodes to 0/1
# rounded_nodes = [round(float(n.value)) for n in nodes]
rounded_nodes = [1 if float(n.value) >= 0.4999 else 0 for n in nodes]

# Update optimal value
new_opt = sum(rounded_nodes)

print(f"Objective value: {round(new_opt, 2)}")
print(f"Solution vector: {[n for n in rounded_nodes]}")

Objective value (before rounding): 50.0
Solution vector (before rounding): [0.500000000027591, 0.49999999999986944, 0.5000000000044335, 0.49999999999428957, 0.5000000000030659, 0.5000000000068782, 0.5000000000071365, 0.5000000000034065, 0.49999999999804845, 0.49999999999884565, 0.49999999998820593, 0.49999999999593014, 0.5000000000206385, 0.49999999999652, 0.49999999999231026, 0.5000000000046911, 0.5000000000021066, 0.49999999999576794, 0.4999999999753837, 0.5000000000007743, 0.500000000018824, 0.49999999999131567, 0.5000000000028995, 0.4999999999978037, 0.4999999999993328, 0.499999999995314, 0.5000000000033956, 0.5000000000066869, 0.5000000000020646, 0.5000000000053395, 0.5000000000242643, 0.4999999999894045, 0.49999999999095024, 0.5000000000024466, 0.5000000000105581, 0.5000000000040499, 0.4999999999942369, 0.5000000000039171, 0.5000000000016301, 0.500000000003655, 0.49999999999829114, 0.5000000000174603, 0.5000000000045337, 0.500000000001898, 0.5000000000172209, 0.4999999999968136, 